In [1]:
import requests
import pandas as pd
import numpy as np
from sodapy import Socrata

In [2]:
# Use API to get paid parking transaction data (https://data.seattle.gov/Transportation/Paid-Parking-Transaction-Data/gg89-k5p6)
client = Socrata("data.seattle.gov", None,timeout=1000000000)

# Find the average duration in minutes, amount paid, and number of transactions for each elementkey
results = client.get_all('gg89-k5p6',select='elementkey,avg(durationinminutes),avg(amount_paid),count(transaction_id)',group='elementkey')
results_df2 = pd.DataFrame.from_records(results)

In [3]:
# Save to new dataframe so I can manipulate data without needing to pull again
results_df = results_df2
results_df

,elementkey,avg_durationinminutes,avg_amount_paid,count_transaction_id
0,1001,80.5762711864406780,3.6281355932203390,59
1,1002,77.0842105263157895,3.3896842105263158,95
2,1005,68.5405405405405405,2.3781081081081081,37
3,10054,72.8695652173913043,2.6407246376811594,69
4,1006,82.7769784172661871,2.5450359712230216,139
...,...,...,...,...
1496,31062,73.0169491525423729,3.7903389830508475,59
1497,35942,134.3913043478260870,2.7234782608695652,46
1498,7266,113.3333333333333333,2.2244444444444444,9
1499,81073,87.7272727272727273,4.7563636363636364,11


In [4]:
# Convert column to int64 format
results_df['elementkey'] = results_df['elementkey'].astype(np.int64)

In [5]:
# Import blockface data (downloaded from https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::blockface/about)
geo_parking = pd.read_csv('Blockface.csv')

In [6]:
# Merge using element keys (left join)
merged_parking = pd.merge(left=geo_parking,right=results_df,how='left',left_on='ELMNTKEY',right_on='elementkey')

In [7]:
merged_parking

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,END_TIME_SAT,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_Length,elementkey,avg_durationinminutes,avg_amount_paid,count_transaction_id
0,1,40194,17631,13640,290,NE 80TH ST BETWEEN 29TH AVE NE AND DEAD END 2,S,NaN,2900,N,...,NaN,DISTRICT4,,N,NaN,31.857765,NaN,NaN,NaN,NaN
1,2,2766,2741,930,380,19TH AVE SW BETWEEN SW CHARLESTOWN E ST AND SW...,E,NaN,3800,N,...,NaN,DISTRICT1,,N,NaN,623.721354,NaN,NaN,NaN,NaN
2,3,37209,14676,11485,160,E PROSPECT ST BETWEEN 16TH E AVE E AND 17TH AVE E,N,NaN,1600,N,...,NaN,DISTRICT3,,N,NaN,290.008049,NaN,NaN,NaN,NaN
3,5,85425,17479,13560,350,NE 73RD ST BETWEEN DEAD END AND 38TH W AVE NE,N,NaN,3500,N,...,NaN,DISTRICT4,,N,NaN,458.683999,NaN,NaN,NaN,NaN
4,6,37217,14681,11485,210,E PROSPECT ST BETWEEN 21ST AVE E AND 22ND AVE E,N,NaN,2100,N,...,NaN,DISTRICT3,,N,NaN,270.891866,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48179,72018,121364,164794,16695,352,S ORCAS ST BETWEEN 35TH E AVE S AND M L KING J...,S,NaN,3500,N,...,NaN,DISTRICT2,,N,NaN,167.862488,NaN,NaN,NaN,NaN
48180,72020,84398,16493,12975,233,NE 125TH ST BETWEEN 24TH W AVE NE AND 24TH E A...,S,NaN,2400,N,...,NaN,DISTRICT5,,N,NaN,186.550990,NaN,NaN,NaN,NaN
48181,72022,21577,21311,16970,4,S SPOKANE NR ST BETWEEN EAST MARGINAL ER WAY S...,N,NaN,0,N,...,NaN,DISTRICT2,,N,NaN,636.805900,NaN,NaN,NaN,NaN
48182,72024,49789,4501,2095,1200,3RD AVE NW BETWEEN NW 120TH ST AND NW 122ND ST,W,NaN,12000,N,...,NaN,DISTRICT5,,N,NaN,642.276544,NaN,NaN,NaN,NaN


In [8]:
# Merge using element keys (inner join)
merged_parking_inner = pd.merge(left=geo_parking,right=results_df,how='inner',left_on='ELMNTKEY',right_on='elementkey')

In [9]:
merged_parking_inner

,OBJECTID,ELMNTKEY,SEGKEY,UNITID,UNITID2,UNITDESC,SIDE,BLOCK_ID,BLOCK_NBR,CSM,...,END_TIME_SAT,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_Length,elementkey,avg_durationinminutes,avg_amount_paid,count_transaction_id
0,34,8405,8293,4790,80,8TH AVE N BETWEEN VALLEY ST AND ALOHA ST,W,N08-08,800,Y,...,06PM,DISTRICT7,,N,NaN,277.142212,8405,153.4400000000000000,4.1224000000000000,75
1,99,53545,8292,4790,70,8TH AVE N BETWEEN ROY ST AND VALLEY ST,W,N08-07,700,Y,...,06PM,DISTRICT7,,N,NaN,269.199277,53545,192.6000000000000000,5.1120000000000000,20
2,243,76197,8288,4790,20,8TH AVE N BETWEEN JOHN ST AND THOMAS ST,W,N08-02,200,Y,...,06PM,DISTRICT7,,N,NaN,400.540628,76197,76.9740259740259740,2.8370129870129870,154
3,271,8622,8527,4890,220,9TH AVE BETWEEN WESTLAKE AVE AND DENNY WAY,NE,09-22,2200,Y,...,06PM,DISTRICT7,,N,NaN,124.450178,8622,43.2783505154639175,2.3730927835051546,97
4,542,24041,1015,10,230,1ST AVE BETWEEN BELL ST AND BATTERY ST,SW,01-23,2300,Y,...,08PM,DISTRICT7,,N,NaN,399.110878,24041,133.3209876543209877,2.5976543209876543,162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,71816,2054,1997,545,152,15TH AVE BETWEEN E MADISON ST AND E PINE ST,E,E15-15,1500,Y,...,10PM,DISTRICT3,,N,NaN,247.531896,2054,76.1789473684210526,3.4610526315789474,95
1497,71816,2054,1997,545,152,15TH AVE BETWEEN E MADISON ST AND E PINE ST,E,E15-15,1500,Y,...,10PM,DISTRICT3,,N,NaN,247.531896,2054,76.1789473684210526,3.4610526315789474,95
1498,71894,34502,11886,8675,20,MINOR AVE N BETWEEN JOHN ST AND THOMAS ST,E,NMR-02,200,Y,...,06PM,DISTRICT7,,N,NaN,393.139890,34502,137.6964285714285714,5.2437500000000000,56
1499,71922,34509,11888,8675,40,MINOR AVE N BETWEEN HARRISON ST AND REPUBLICAN ST,W,NMR-04,400,Y,...,06PM,DISTRICT7,,N,NaN,390.762864,34509,131.2222222222222222,5.0142592592592593,54
